In [27]:
import numpy as np
import nevergrad as ng
from scipy.stats import multivariate_normal

# Functions to be tested

For now, use a very simple function - quadratic with max value 0, and x* = [0.5, 0.5] 

Also note that we will maximize the function, not minimize.

In [28]:
def F(theta):
    return -sum((theta - 0.5) ** 2)

In [29]:
def ES_benchmark_gradient(alpha, sigma, theta_0, num_samples, time_steps):
    theta_t = theta_0
    d = theta_0.shape[0]
    n = num_samples
    for t in range(time_steps):
        #**** sample epsilons ****#
        eps_list = [] 
        for i in range(n):
            eps = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d))
            eps_list.append(eps)
        #**** compute function values ****#
        F_list = []
        for i in range(n):
            F_val = F(theta_t + sigma*eps_list[i])
            F_list.append(F_val)
        #**** update theta ****#
        new_theta = theta_t
        for i in range(n):
            new_theta += alpha / (n*sigma) * F_list[i] * eps_list[i]
        theta_t = new_theta
    return theta_t, F(theta_t)

In [30]:
ES_benchmark_gradient(alpha=0.001, sigma=0.01, theta_0 = np.array([1.0,1.0]), num_samples = 10, time_steps = 1000)

(array([0.49988271, 0.50019313]), -5.1055882571687483e-08)

In [106]:
def ES_hessian(alpha, sigma, theta_0, mu, num_samples, time_steps, p):
    theta_t = theta_0
    d = theta_0.shape[0]
    n = num_samples
    H = None
    for t in range(time_steps):
        #**** sample epsilons ****#
        eps_list = [] 
        for i in range(n):
            eps = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d))
            eps_list.append(eps)
        #**** compute function values ****#
        F_list = []
        for i in range(n):
            F_val = F(theta_t + sigma*eps_list[i])
            F_list.append(F_val)
        #**** compute Hessian ****#
        if t % p == 0:
            H = np.zeros((d,d))
            for i in range(n):
                e_i = eps_list[i].reshape((d,1))
                e_i_trans = eps_list[i].reshape((1,d))
                H += F_list[i] * (np.matmul(e_i, e_i_trans) - np.identity(d) ) / n
            H /= sigma**2
        #**** update theta: compute g ****#
        u, s, vh = np.linalg.svd(H)
        H_nh = u @ np.diag(s**-0.5)
        g = 0
        for i in range(n):
            e_i = eps_list[i].reshape((d,1)) 
            F_new = F(theta_t +  mu * (H_nh @ e_i)  )
            g += ((F_new - F(theta_t)) / mu ) @ (H_nh @ e_i) / n
        #**** update theta: the rest ****#
        new_theta = theta_t + alpha * g
        theta_t = new_theta
        
    return theta_t, F(theta_t), H

In [109]:
ES_hessian(alpha=0.005, sigma=0.01, theta_0 = np.array([1.0,1.0]), mu = .5, num_samples = 50, time_steps = 6000, p = 10)

(array([0.50649885, 0.50649885]),
 -8.44700091230673e-05,
 array([[-1.26667442,  1.54922622],
        [ 1.54922622, -4.3679784 ]]))